In [1]:
%autosave 0
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosave disabled


# Evaluation of Formulas from First Order Logic

In this notebook we show how formulas from first order logic can be evaluated in Python.

## The Axioms of Group Theory

To have a nontrivial example of formulas, we use the formulas from *group theory*.  A *group* is defined as a triple 
$$ \langle G, \mathrm{e}, * \rangle $$
where 
<ol>
    <li>$G$ is a set,</li>
    <li>$\mathrm{e}$ is an element from $G$, and</li>
    <li>$*:G \times G \rightarrow G$ is a binary function on $G$.</li>
    <li>Furthermore, the following axioms have to be satisfied:
        <ul>
            <li>$\forall x: \mathrm{e} * x = x$,</li>
            <li>$\forall x: \exists{y}: y * x = \mathrm{e}$,</li>
            <li>$\forall x: \forall y: \forall z: (x * y) * z = x * (y * z)$.</li>
        </ul>
    </li>
    <li>A group is <em>commutative</em> if, additionally, the following formula is satisfied:
        <ul>
            <li>$\forall x: \forall y: x * y = y * x$.</li>
        </ul>
</ol>

In [2]:
import folParser as fp

Our parser distinguishes variables and function symbol as follows:
<ul>
    <li>A word starting with a lower case letter is interpreted as a *variable*.</li>
    <li>A word starting with an upper case letter is assumed to be a function or predicate symbol.</li>
</ul>

Therefore, we represent the symbols from group theory as follows:
<ol>
    <li>The neutral element $\mathrm{e}$ of group theory is represented as the nullary function symbol $\texttt{E}$.</li>
    <li>As our parser does not support using the symbol $*$ as a binary operator, we will use the function symbol <tt>Multiply</tt> to represent this operator.</li>
    <li>The predicate symbol $=$ is repesented as $\texttt{Equals}$.</li>
</ol>
Then the formulas of group theory can be represented as follows:

In [3]:
G1 = '∀x:Equals(Multiply(E(),x),x)'

In [4]:
G2 = '∀x:∃y:Equals(Multiply(x,y),E())'

In [5]:
G3 = '∀x:∀y:∀z:Equals(Multiply(Multiply(x,y),z), Multiply(x,Multiply(y,z)))'

In [6]:
G4 = '∀x:∀y:Equals(Multiply(x,y), Multiply(y,x))'

The function $\texttt{parse}(s)$ takes a string $s$ and converts it into a nested tuple.

In [7]:
def parse(s):
    "Parse string s as fol formula."
    p = fp.LogicParser(s)
    return p.parse()

In [8]:
F1 = parse(G1)
F1

('∀', 'x', ('Equals', ('Multiply', ('E',), 'x'), 'x'))

In [9]:
F2 = parse(G2)
F2

('∀', 'x', ('∃', 'y', ('Equals', ('Multiply', 'x', 'y'), ('E',))))

In [10]:
F3 = parse(G3)
F3

('∀',
 'x',
 ('∀',
  'y',
  ('∀',
   'z',
   ('Equals',
    ('Multiply', ('Multiply', 'x', 'y'), 'z'),
    ('Multiply', 'x', ('Multiply', 'y', 'z'))))))

In [11]:
F4 = parse(G4)
F4

('∀',
 'x',
 ('∀', 'y', ('Equals', ('Multiply', 'x', 'y'), ('Multiply', 'y', 'x'))))

## A Structure for Group Theory

The smallest group has just two elements.  We define these elements as the strings <tt>"a"</tt> and  <tt>"b"</tt>.  Since we do not want to type the double quote symbols every time we use these elements, we store these in the variables  <tt>a</tt> and <tt>b</tt>.  Then, we can define the universe <tt>U</tt> as follows:

In [12]:
a = "a"
b = "b"
U = { a, b }  

Next, we need to define the nullary function that represents the nullary function <tt>E</tt>.  We define this function as a dictionary mapping the empty tuple into the element <tt>a</tt>. 

In [13]:
NeutralElement = { (): a }

The binary function symbol <tt>Multiply</tt> is implemented as the dictionary <tt>Product</tt>:

In [14]:
Product = { (a, a): a,  (a, b): b,  (b, a): b,  (b, b): a }

The predicate symbol <tt>Equals</tt> is implemented as the binary relation <tt>Identity</tt>.

In [15]:
Identity = { (x, x) for x in U }

Now the interpretation $\mathcal{J}$ can be implemented as a dictionary.

In [16]:
J = { "E": NeutralElement, "Multiply": Product, "Equals": Identity }

Next, we define the first order structure $\mathcal{S}$.

In [17]:
S = (U, J)

Finally, we define an interpretation $\mathcal{I}$ of the variables $x$, $y$, and $z$. 

In [18]:
I = { "x": a, "y": b, "z": a } 

## Functions to Evaluate Formulas

The procedure $\texttt{evalTerm}(t, S, I)$ evaluates the term $t$ in the structure $\mathcal{S}$ using the variable assignment $\mathcal{I}$.

In [19]:
def evalTerm(t, S, I):
    if isinstance(t, str):  # t is a variable
        return I[t]
    J        = S[1]   # dictionary of interpretations
    f        = t[0]   # function symbol
    fJ       = J[f]   # interpretation of function symbol
    argTuple = t[1:]
    argVals  = evalTermTuple(argTuple, S, I)
    return fJ[argVals]

The procedure $\texttt{evalTermTuple}(\texttt{Ts}, \mathcal{S}, \mathcal{I})$ evaluates the tuple of terms $\texttt{Ts}$, given the structure $\mathcal{S}$ and the variable assignment $\mathcal{I}$.

In [20]:
def evalTermTuple(Ts, S, I):
    return tuple(evalTerm(t, S, I) for t in Ts)

In [21]:
t = parse('Multiply(E(),x)')
t

('Multiply', ('E',), 'x')

In [22]:
evalTerm(t, S, I)

'a'

This procedure evaluates the atomic formula a in the structure S using the variable assignment I.

In [23]:
def evalAtomic(a, S, I):
    J        = S[1]  # dictionary of interpretations
    p        = a[0]  # predicate symbol
    pJ       = J[p]  # interpretation of predicate symbol
    argTuple = a[1:]
    argVals  = evalTermTuple(argTuple, S, I)
    return argVals in pJ

In [24]:
f = parse('Equals(Multiply(E(),x),x)')
f

('Equals', ('Multiply', ('E',), 'x'), 'x')

In [25]:
evalAtomic(f, S, I)

True

Given a variable assignment $\mathcal{I}$, a variable $x$, and an element $c$ from the universe $\mathcal{U}$, the function $\texttt{modify}(\mathcal{I}, x, c)$ computes the variable assignement $\mathcal{I}[x/c]$ which is defined for all variables $y$ as follows:
$$ I[x/c](y) = \left\{ \begin{array}{ll}
                        c     & \mbox{if $x = y$,}  \\
                        I(y)  & \mbox{otherwise.}
                        \end{array}
               \right.
$$

In [26]:
def modify(I, x, c):
    I[x] = c
    return I

Given a first order logic formula $F$, a structure $\mathcal{S}$, and a variable assignment $\mathcal{I}$, the function $\texttt{evalFormula}(F, \mathcal{S}, \mathcal{I})$ computes the truth value of the formula $F$.

In [27]:
def evalFormula(F, S, I):
    U = S[0]  # universe
    if F[0] == '⊤': return True
    if F[0] == '⊥': return False
    if F[0] == '¬': return not evalFormula(F[1], S, I)
    if F[0] == '∧': return evalFormula(F[1], S, I) and evalFormula(F[2], S, I)
    if F[0] == '∨': return evalFormula(F[1], S, I) or evalFormula(F[2], S, I)
    if F[0] == '→': return not evalFormula(F[1], S, I) or evalFormula(F[2], S, I)
    if F[0] == '↔': return evalFormula(F[1], S, I) == evalFormula(F[2], S, I)
    if F[0] == '∀': 
        x, G = F[1:] 
        return all({ evalFormula(G, S, modify(I, x, c)) for c in U })
    if F[0] == '∃':
        x, G = F[1:] 
        return any({ evalFormula(G, S, modify(I, x, c)) for c in U })
    return evalAtomic(F, S, I) 

## Checking whether $\mathcal{S}$ is a Group

In [28]:
print(f"evalFormula({G1}, S, I) = {evalFormula(F1, S, I)}")
print(f"evalFormula({G2}, S, I) = {evalFormula(F2, S, I)}")
print(f"evalFormula({G3}, S, I) = {evalFormula(F3, S, I)}")
print(f"evalFormula({G4}, S, I) = {evalFormula(F4, S, I)}")

evalFormula(∀x:Equals(Multiply(E(),x),x), S, I) = True
evalFormula(∀x:∃y:Equals(Multiply(x,y),E()), S, I) = True
evalFormula(∀x:∀y:∀z:Equals(Multiply(Multiply(x,y),z), Multiply(x,Multiply(y,z))), S, I) = True
evalFormula(∀x:∀y:Equals(Multiply(x,y), Multiply(y,x)), S, I) = True


This shows that the structure $\mathcal{S}$ defined above is indeed a group.  Furthermore, the law of commutativity holds true in this group.